理解随机过程的含义

赌博中的随机过程

赌博的例子非常普遍，这里我们举一个大家非常熟悉的例子，我们尽量挖掘的更深入一些。

赌徒和庄家对赌抛硬币，如果为正面，本轮赌徒赢，庄家付给赌徒1元，结果为反面，本轮赌徒输，赌徒付给庄家1元。赌徒有初始赌本10元，手上的钱一旦输光则退出赌局，如何来模拟这个赌博过程？

我们首先来分析一下这个过程，赌徒的赌博结果本质上依托于每次抛掷硬币的结果，每一轮赌博就是一个伯努利试验，赢的概率是p=0.5，赌博的过程就是由这一串伯努利试验构成的伯努利随机过程，每轮赌局中，如果赢则赌本增加1元，输则赌本减少1元。

当然了，如果对某一个特定的赌徒，一旦开始进入赌局，则最终由每轮赌局结果构成的序列就是唯一的。那么如果我们想观察整个赌博过程的整体特征，我们该怎么办？好办，还是使用之前讲过的蒙特卡罗方法，采用大量的样本，最终观察样本结果的整体特征。

这里，我们为了说明问题，先采用的样本数也就是赌徒数为10个，轮数为100轮，也就是每个赌徒最多和庄家对赌100轮，如果在这个过程中输光了赌本，则提前退出，如果到100轮还有赌本，赌局也停止。

我们来看一下模拟的代码：

In [2]:
import pandas as pd
import random

sample_list = []
round_num =100
person_num = 10
for person in range (1,person_num + 1):
    money = 10 #本金十块
    for round in range(1,round_num + 1):
        result = random.randint(0, 1) #随机输赢
        if result == 1:
            money = money + 1
        elif result == 0:
            money = money - 1
        if money == 0:
            break
    sample_list.append([person,round,money])


sample_df = pd.DataFrame(sample_list,columns = ['person','round','money'])
sample_df.set_index('person',inplace=True)


# print(sample_df)
pd.DataFrame(sample_df)

,round,money
person,,
1,100,4
2,100,10
3,100,10
4,48,0
5,100,16
6,100,4
7,100,18
8,100,22
9,100,24
